### Import the data set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/9. Artificial Intelligence/3. Presentations/AI PPT Materials Day 1-30/Day 20/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/9. Artificial Intelligence/3. Presentations/AI PPT Materials Day 1-30/Day 20/test.csv")
train_df.head()

### # How many samples of each class?

In [ ]:
train_df.target.value_counts()

In [ ]:
train_df_shuffled = train_df.sample(frac=1,
                                    random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

### Let's visualize some random training examples

In [ ]:
import random
random_index = random.randint(0,
                              len(train_df)-5) # create random indexes not higher than the total number of samples

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}",
        "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

In [ ]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42) # random state for reproducibility

In [ ]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization"

# Use the default TextVectorization variables
text_vect = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?


In [ ]:
# Fit the text vectorizer to the training text
text_vect.adapt(train_sentences)

In [ ]:
# Create sample sentence and tokenize it
sample_sentence = "There is flood in my city"
text_vect([sample_sentence])

In [ ]:
# Create sample sentence and tokenize it
sample_sentence = "There is flood in my city and we are looking for help"
text_vect([sample_sentence])

In [ ]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

In [ ]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create sample sentence and tokenize it
sample_sentence = "There is flood in my city"
text_vectorizer([sample_sentence])

In [ ]:
# Create sample sentence and tokenize it
sample_sentence = "There is flood in my city and we are looking for help"
text_vectorizer([sample_sentence])

## DRAWBACKS of Textvectorization:
##           1. creats very huge matrix
##           2. results in sparse matrix representation
##           3. provides static vector representation

## Word Embedding

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")



In [ ]:
sample_sentence = "There is flood in my city"
sample_embed = embedding(text_vectorizer([sample_sentence]))
sample_embed

In [ ]:
# Check out a single token's embedding
sample_embed[0][0]

In [ ]:
# Create LSTM model
inputs = layers.Input(shape=(1,),
                      dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(64,
                 activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1,
                       activation="sigmoid")(x)
model = tf.keras.Model(inputs,
                       outputs,
                       name="model_2_LSTM")

In [ ]:
# Compile model
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit model
model_history = model.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

In [ ]:
# Make predictions on the validation dataset
model_pred_probs = model.predict(val_sentences)
model_pred_probs.shape, model_pred_probs[:10] # view the first 10

In [ ]:
### We can turn these prediction probabilities into prediction classes by rounding to the nearest integer
### (by default, prediction probabilities under 0.5 will go to 0 and those over 0.5 will go to 1).

# Round out predictions and reduce to 1-dimensional array
model_preds = tf.squeeze(tf.round(model_pred_probs))
model_preds[:10]

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_acc = accuracy_score(val_labels, model_preds) * 100
model_acc

### Model 2: GRU

* Another popular and effective RNN component is the GRU or gated recurrent unit.

* The GRU cell has similar features to an LSTM cell but has less parameters.

In [ ]:
inputs = layers.Input(shape=(1,),
                      dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x) # return vector for whole sequence
x = layers.Dense(64,
                 activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1,
                       activation="sigmoid")(x)
model = tf.keras.Model(inputs,
                       outputs,
                       name="model_2_LSTM")

In [ ]:
# Compile GRU model
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_history = model.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

In [ ]:
# Make predictions on the validation dataset
model_pred_probs = model.predict(val_sentences)
model_pred_probs.shape, model_pred_probs[:10]

In [ ]:
model_preds = tf.squeeze(tf.round(model_pred_probs))
model_preds[:10]

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_acc = accuracy_score(val_labels, model_preds) * 100
model_acc

### Model 3: Bidirectonal RNN model

* A standard RNN will process a sequence from left to right, where as a bidirectional RNN will process the sequence from left to right and then again from right to left.
 * In practice, many sequence models often see and improvement in performance when using bidirectional RNN's.

* However, this improvement in performance often comes at the cost of longer training times and increased model parameters (since the model goes left to right and right to left, the number of trainable parameters doubles).

In [ ]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,),
                      dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # stacking RNN layers requires return_sequences=True
x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1,
                       activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,
                         outputs,
                         name="model_4_Bidirectional")

In [ ]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model (takes longer because of the bidirectional layers)
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

In [ ]:
# Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
# Convert prediction probabilities to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_acc = accuracy_score(val_labels, model_4_preds) * 100
model_acc